## Generate Static Graphs

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [ ]:
year = 2021
month = 9
program = "prepilot"
study_type = "program"
mode_of_interest = "Pilot ebike"

In [ ]:
from collections import defaultdict
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import emission.core.get_database as edb
import emission.core.wrapper.entry as ecwe
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.storage.decorations.trip_queries as esdt
import emission.storage.decorations.timeline as esdl
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.timequery as estt
from plots import *
import scaffolding
from uuid import UUID

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

In [ ]:
# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r df_ei
%store -r dic_re
%store -r dic_pur
%store -r dic_fuel

# convert a dictionary to a defaultdict
dic_re = defaultdict(lambda: 'Other',dic_re)
dic_pur = defaultdict(lambda: 'Other',dic_pur)
dic_fuel = defaultdict(lambda: 'Other',dic_fuel)

## Collect Data From Database

In [ ]:
tq = scaffolding.get_time_query(year, month)
participant_ct_df = scaffolding.load_all_participant_trips(program, tq)
labeled_ct = scaffolding.filter_labeled_trips(participant_ct_df)
expanded_ct = scaffolding.expand_userinputs(labeled_ct)
expanded_ct = scaffolding.data_quality_check(expanded_ct)

In [ ]:
# Mapping new labels with dictionaries
expanded_ct['Trip_purpose']= expanded_ct['purpose_confirm'].map(dic_pur)
expanded_ct['Mode_confirm']= expanded_ct['mode_confirm'].map(dic_re)
expanded_ct['Replaced_mode']= expanded_ct['replaced_mode'].map(dic_re)

# Mapping fuel
expanded_ct['Mode_confirm_fuel']= expanded_ct['Mode_confirm'].map(dic_fuel)
expanded_ct['Replaced_mode_fuel']= expanded_ct['Replaced_mode'].map(dic_fuel)

# Change meters to miles
scaffolding.unit_conversions(expanded_ct)

# Calculate energy impact
expanded_ct = scaffolding.energy_intensity(expanded_ct, df_ei, 'distance_miles', 'Replaced_mode', 'Mode_confirm')
expanded_ct = scaffolding.energy_impact_kWH(expanded_ct, 'distance_miles', 'Replaced_mode', 'Mode_confirm')
expanded_ct = scaffolding.CO2_impact_lb(expanded_ct, 'distance_miles', 'Replaced_mode', 'Mode_confirm')

In [ ]:
file_suffix = scaffolding.get_file_suffix(year, month, program)
quality_text = scaffolding.get_quality_text(participant_ct_df, expanded_ct)

## Mode Shares

In [ ]:
# Count of trips per mode
mode_confirm_vals_df = pd.DataFrame(expanded_ct['mode_confirm'].value_counts())
total_mode_confirms = mode_confirm_vals_df.sum()
top_ten_mode_confirm = mode_confirm_vals_df.iloc[0:9].copy()

# All modes not in top 10 are labeled misc
misc_count = (total_mode_confirms - top_ten_mode_confirm.sum())
top_ten_mode_confirm.loc["misc"] = misc_count
top_ten_mode_confirm

In [ ]:
pctFmt = lambda p: '%.1f%%'%p if p > 5 else None

In [ ]:
ax = top_ten_mode_confirm.mode_confirm.plot.pie(autopct=pctFmt)
ax.set_ylabel("")
ax.set_title("Travel modes selected by users\n%s" % quality_text)

In [ ]:
ax.figure.savefig("/plots/mode_confirm%s.png" % file_suffix)
alt_text = scaffolding.store_alt_text_pie(top_ten_mode_confirm, 'mode_confirm', 'travel modes', './alt_text_lookup.json')

In [ ]:
below_top_ten_mode_confirm = mode_confirm_vals_df.iloc[9:]
below_top_ten_mode_confirm

In [ ]:
ax = below_top_ten_mode_confirm.mode_confirm.plot.pie(autopct=pctFmt)
ax.set_ylabel("")
ax.set_title("Expansion of the 'misc' travel modes\n%s" % quality_text)

In [ ]:
ax.figure.savefig("/plots/mode_confirm_misc_expand%s.png" % file_suffix)
alt_text = scaffolding.store_alt_text_pie(below_top_ten_mode_confirm, 'mode_confirm_misc_expand', 'travel modes', './alt_text_lookup.json')